In [ ]:
#https://keras.io/examples/generative/vae/
#https://keras.io/examples/generative/molecule_generation/

In [10]:
import os
from sklearn.preprocessing import MinMaxScaler
import joblib
import pandas as pd
#================== initialization ==================
LATENT_DIM = 128
VAE_LR = 5e-4
PARQUET_PATH = './data/OptionsEOD_STG.parquet'
SCALER_PATH = './data/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['DTE','INTRINSIC_VALUE','TOTAL_C_VOLUME', 'TOTAL_P_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]

In [11]:
#Example
import random
from IPython.display import clear_output,display, HTML

#load scaler
scaler = MinMaxScaler()
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
random.shuffle(PartitionDate)
for i,partdate in enumerate(PartitionDate) :
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    key = random.choice( [*df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates().values] )
    df_filtered=df[ (df['QUOTE_DATE'] == key[0]) 
        & (df['SYMBOL'] == key[1]) 
        & (df['EXPIRE_DATE'] == key[2]) 
    ]
    break
scaler = joblib.load(SCALER_PATH)
df_tf = scaler.transform(df_filtered[SCALER_COL])

In [12]:
df_filtered[SCALER_COL]

,DTE,INTRINSIC_VALUE,TOTAL_C_VOLUME,TOTAL_P_VOLUME,C_BID,C_ASK,C_VOLUME,P_BID,P_ASK,P_VOLUME
13735,51.0,-145.52,329.0,6.0,6.10,6.69,19.0,151.70,162.79,0.0
13736,51.0,-150.52,329.0,6.0,5.40,6.00,56.0,156.20,167.51,3.0
13737,51.0,-155.52,329.0,6.0,4.80,5.40,41.0,160.19,171.60,0.0
13738,51.0,-160.52,329.0,6.0,4.29,4.90,8.0,165.00,176.40,0.0
13739,51.0,-165.52,329.0,6.0,3.81,4.40,22.0,169.00,180.60,0.0
13740,51.0,-170.52,329.0,6.0,3.30,3.90,0.0,173.60,185.80,1.0
13741,51.0,-175.52,329.0,6.0,2.90,3.51,4.0,178.20,190.09,0.0
13742,51.0,-180.52,329.0,6.0,2.55,3.10,2.0,182.80,197.01,0.0
13743,51.0,-185.52,329.0,6.0,2.20,2.75,7.0,188.00,199.70,0.0
13744,51.0,-190.52,329.0,6.0,1.89,2.46,46.0,192.20,206.00,1.0


In [5]:
df_tf.shape

(17916, 10)

In [7]:
df_tf[0]

array([1.49622950e-03, 4.22338685e-01, 1.51827076e-02, 2.71138759e-02,
       3.46804242e-03, 6.07996960e-05, 0.00000000e+00, 0.00000000e+00,
       4.99997475e-08, 0.00000000e+00])

In [ ]:
from src.model import OptionChainGenerator
from src.layer import encoder, decoder

model = OptionChainGenerator(
    encoder(latent_dim = LATENT_DIM, 
            input_shape= , 
            gconv_units = [64, 128], 
            dense_units = [256, 128], 
            dropout_rate= 0.1,
            output_dim  = 
           ), 
    decoder(latent_dim  = LATENT_DIM , 
            output_shape= ,
            dense_units = [128, 256],
            dropout_rate= 0.1
           )
)
model.summery()
vae_optimizer = tf.keras.optimizers.Adam(learning_rate=VAE_LR)
model.compile(vae_optimizer)

In [ ]:
#================== train model ==================
history = model.fit([feature_tensor], epochs=EPOCHS)


In [ ]:
import numpy as np

def create_adjacency_matrix(options, threshold=25):
    """
    สร้าง adjacency matrix สำหรับ Options Chain โดยใช้ความใกล้เคียงของ strike price และ DTE
    """
    num_options = len(options)
    adjacency_matrix = np.zeros((num_options, num_options))

    for i in range(num_options):
        for j in range(i, num_options):
            # พิจารณาเชื่อมโยงระหว่าง options หาก strike price ต่างกันไม่เกิน threshold และ DTE เท่ากัน
            if abs(options[i]['strike_price'] - options[j]['strike_price']) <= threshold and options[i]['dte'] == options[j]['dte']:
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1
    
    return adjacency_matrix

options = [
    {'strike_price': 100, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 115, 'dte': 30},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 0, 'dte': 0},  # เปลี่ยนค่าเป็น 0
]

options = [
    {'strike_price': 0, 'dte': 0},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 150, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 100, 'dte': 30},  # เปลี่ยนค่าเป็น 0
]


# ปรับ threshold
threshold = 5

adjacency_matrix = create_adjacency_matrix(options, threshold)
print(adjacency_matrix)

In [ ]:
import numpy as np
A = np.array(
    [[1,2,3],
    [4,5,6],
    [7,8,9]]
    
)

In [ ]:
A[:,:,0]

In [ ]:
def get_variable_name(variable):
    for name, value in globals().items():
        if value is variable:
            return name
    return None

# Example usage
leo = ['a', 'b']
var_name = get_variable_name(leo)
print(var_name)  # Ou